In [1]:
import pandas as pd
import os

In [3]:
# 设置文件路径，导入路径
folder_path = r"E:\Test\Inputs"


# file_coalfpplant = os.path.join(folder_path, "coal-fired_power_plants_China_2020_operating.xlsx")
# df_coalfpplant = pd.read_excel(file_coalfpplant)

# file_coal2 = os.path.join(folder_path, "coal-fired_power_plants_China_wihout_ot_2020_operating.xlsx")
# df_coal2 = pd.read_excel(file_coal2)

# file_wasteinplant = os.path.join(folder_path, "waste_inceration_plants_2020_operating.xlsx")
# df_wasteinplant = pd.read_excel(file_wasteinplant)

# file_gas_plant = os.path.join(folder_path, "gas-fired_power_plants_China_2020_operating.xlsx")
# df_gas_plant = pd.read_excel(file_gas_plant)

# file_coalcon = os.path.join(folder_path, "coal_consumption_2020.xlsx")
# df_coalcon = pd.read_excel(file_coalcon)

file_thermal_gen = os.path.join(folder_path,"province_thermal_power_generation_2020.xlsx")
df_thermal_gen = pd.read_excel(file_thermal_gen)

# file_biomass = os.path.join(folder_path, "province_biomass_power_generation_2020.xlsx")
# df_biomass = pd.read_excel(file_biomass)

file_load = os.path.join(folder_path, "load_profile_monthly_province.csv")
df_load = pd.read_csv(file_load)

# file_load = os.path.join(folder_path, "heat_profile_monthly_province.csv")
# df_load = pd.read_csv(file_load)

# file_water_int = os.path.join(folder_path,"province_intensity.xlsx")
# df_water_int = pd.read_excel(file_water_int)

In [8]:
df_CFPP = pd.read_excel(os.path.join(folder_path,"CFPP.xlsx"))
df_CFPP_2 = pd.read_excel(os.path.join(folder_path,"CFPP_WITHOUT_OT.xlsx"))
df_WIPP = pd.read_excel(os.path.join(folder_path,"WIPP.xlsx"))
df_GFPP = pd.read_excel(os.path.join(folder_path,"GFPP.xlsx"))

## 计算电厂年发电/供热量（根据省年发电/供热量、厂装机容量）

In [3]:
heat_supply_p = dict(zip(df_water_int['Province'],df_water_int['heat_supply_104GJ']))
capa_p = dict(zip(df_thermal_gen['province'],df_thermal_gen['capacity_total_10MW']))

In [3]:
## 计算每个垃圾焚烧厂的年发电量(kWh)、供热量(GJ)

## 每个垃圾焚烧厂的年发电量 = 省垃圾焚烧发电总量（1e8 kWh）* 垃圾焚烧厂装机容量（MW）/ 省垃圾焚烧厂发电总装机容量（10MW）

# 创建一个字典，将province作为键，capacity_wastein和gen_wastein作为值
provincebio_dict = dict(zip(df_biomass['province'], zip(df_biomass['capacity_wastein'], df_biomass['gen_wastein'])))

# 定义一个函数来计算Annual_Gen
def calculate_annual_gen(row):
    province = row['Pro_en']
    capacity_wastein, gen_wastein = provincebio_dict[province]
    return row['Capacity_elec_MW'] / capacity_wastein / 10 * gen_wastein * 1e8

# 在df_wasteinplant中新增一列Annual_Gen_kWh
df_wasteinplant['Annual_Gen_kWh'] = df_wasteinplant.apply(calculate_annual_gen, axis=1)

df_wasteinplant.to_excel(os.path.join(folder_path, "waste_inceration_plants_2020_operating.xlsx"), index=False)

In [5]:
## 计算每个煤电厂的年发电量(kWh)

## 每个煤电厂的年发电量 = 省煤电发电总量（1e8 kWh）* 煤电厂装机容量（MW）/ 省煤电厂发电总装机容量（10MW）

# 创建一个字典，将province作为键，capacity_wastein和gen_wastein作为值
provincecoal_dict = dict(zip(df_thermal_gen['province'], zip(df_thermal_gen['capacity_coal'], df_thermal_gen['gen_coal'])))

# 定义一个函数来计算Annual_Gen
def calculate_annual_gen(row):
    province = row['Province']
    capacity_coal, gen_coal = provincecoal_dict[province]
    return row['Capacity_MW'] / capacity_coal / 10 * gen_coal * 1e8

# 在df_wasteinplant中新增一列Annual_Gen_kWh
df_coalfpplant['Annual_Gen_kWh'] = df_coalfpplant.apply(calculate_annual_gen, axis=1)

df_coalfpplant.to_excel(os.path.join(folder_path, "coal-fired_power_plants_China_2020_operating.xlsx"), index=False)

In [ ]:
## 年供热量(GJ) = 省火电总供热量（1e4 GJ）* 垃圾焚烧厂装机容量（MW）/ 省火电总装机（10MW）

# 定义一个函数来计算Annual_Heat
def calculate_annual(row):
    province = row['Pro_en']
    pro_hs = heat_supply_p[province]
    capa_total = capa_p[province]
    return row['Capacity_elec_MW'] / capa_total / 10 * pro_hs * 1e4

# 在df_wasteinplant中新增一列Annual_Heat_GJ
df_wasteinplant['Annual_Heat_GJ'] = df_wasteinplant.apply(calculate_annual, axis=1)

# 计算Yearly_Water_Heat_m3
df_merged = pd.merge(df_wasteinplant, df_water_int[['Province', 'heat_WC_m3/GJ']], left_on='Pro_en', right_on='Province', how='left')
df_merged['Yearly_Water_Heat_m3'] = df_merged['Annual_Heat_GJ'] * df_merged['heat_WC_m3/GJ']

# 删除df_merged中的heat_WC_m3/GJ列
df_merged = df_merged.drop(columns=['heat_WC_m3/GJ'])
df_merged.to_excel(os.path.join(folder_path, "waste_inceration_plants_2020_operating_with_water_heat.xlsx"), index=False)

In [6]:
## 年煤电供热量(GJ) = 省火电总供热量（1e4 GJ）* 煤电装机（MW）/ 省火电总装机（10MW）

# 定义一个函数来计算Annual_Heat
def calculate_annual(row):
    province = row['Province']
    pro_hs = heat_supply_p[province]
    capa_total = capa_p[province]
    return row['Capacity_MW'] / capa_total / 10 * pro_hs * 1e4

# 新增一列Annual_Heat_GJ
df_coal2['Annual_Heat_GJ'] = df_coal2.apply(calculate_annual, axis=1)

# 计算Yearly_Water_Heat_m3
df_merged = pd.merge(df_coal2, df_water_int[['Province', 'heat_WC_m3/GJ']], left_on='Province', right_on='Province', how='left')
df_merged['Yearly_Water_Heat_m3'] = df_merged['Annual_Heat_GJ'] * df_merged['heat_WC_m3/GJ']
df_merged = df_merged.drop(columns=['heat_WC_m3/GJ'])

df_merged.to_excel(os.path.join(folder_path, "coal-fired_power_plants_China_without_ot_2020_operating.xlsx"), index=False)
# df_coalfpplant['Annual_Heat_GJ'] = df_coalfpplant.apply(calculate_annual, axis=1)

# df_coalfpplant.to_excel(os.path.join(folder_path, "coal-fired_power_plants_China_2020_operating.xlsx"), index=False)

In [7]:
## 年气电供热量(GJ) = 省火电总供热量（1e4 GJ）* 气电装机（MW）/ 省火电总装机（10MW）

# 定义一个函数来计算Annual_Heat
def calculate_annual(row):
    province = row['Province']
    pro_hs = heat_supply_p[province]
    capa_total = capa_p[province]
    return row['Capacity_MW'] / capa_total / 10 * pro_hs * 1e4

# 新增一列Annual_Heat_GJ
df_gas_plant['Annual_Heat_GJ'] = df_gas_plant.apply(calculate_annual, axis=1)

# 计算Yearly_Water_Heat_m3
df_merged = pd.merge(df_gas_plant, df_water_int[['Province', 'heat_WC_m3/GJ']], left_on='Province', right_on='Province', how='left')
df_merged['Yearly_Water_Heat_m3'] = df_merged['Annual_Heat_GJ'] * df_merged['heat_WC_m3/GJ']
df_merged = df_merged.drop(columns=['heat_WC_m3/GJ'])

df_merged.to_excel(os.path.join(folder_path, "gas-fired_power_plants_China_2020_operating.xlsx"), index=False)


## 计算电厂月发电量、燃料量（根据负荷曲线、能量转换）

In [7]:
## 时间精度年→月 计算每个煤电厂的月发电量及发电煤耗，格式同污水厂数据
month_dict = {
    'Jan': '01',
    'Feb': '02',
    'Mar': '03',
    'Apr': '04',
    'May': '05',
    'Jun': '06',
    'Jul': '07',
    'Aug': '08',
    'Sep': '09',
    'Oct': '10',
    'Nov': '11',
    'Dec': '12'
}

df_CFPP_columns = ["Province","Plantcode", "Plantname", "Statistical month", "Capacity_MW", "Cooling_tech", "Combustion_tech", "Capacity_level", "Lifetime", "Monthly_Gen_kWh"]
df_CFPP = pd.DataFrame(columns=df_CFPP_columns)

for index, row in df_coalfpplant.iterrows():
    province = row['Province']
    plantcode = row['Plant_Code']
    plantname = row['Plantname']
    capacity = row['Capacity_MW']
    cooling_tech = row['Cooling_tech']
    combustion_tech = row['Combustion_tech']
    capacity_level = row['Capacity_level']
    lifetime = row['Lifetime']
    annual_gen = row['Annual_Gen_kWh']

    for month in month_dict.keys():
        date_str = "2020" + month_dict[month] #生成日期字符串
        monthly_gen = annual_gen *  df_load.loc[df_load["Province"] == province, month].values[0] #获取特定省份、月份的发电量比例，计算月发电量
        df_CFPP.loc[len(df_CFPP)] = [province, plantcode, plantname, date_str, capacity, cooling_tech, combustion_tech, capacity_level, lifetime, monthly_gen]


### 计算月煤耗量
provincecoalcon_dict = dict(zip(df_coalcon['Province'], df_coalcon['Coal_con(g/kWh)']))
Monthly_Coal = df_CFPP.apply(lambda row: row['Monthly_Gen_kWh'] * provincecoalcon_dict.get(row['Province'], 0), axis=1) / 1e6 #将g转化为吨
df_CFPP['Monthly_Coal_t'] = Monthly_Coal

df_CFPP.to_excel(os.path.join(folder_path, "CFPP.xlsx"), index=False)

In [4]:
## 时间精度年→月 计算每个垃圾焚烧厂的月发电量(kWh)及垃圾焚烧量(t)
month_dict = {
    'Jan': '01',
    'Feb': '02',
    'Mar': '03',
    'Apr': '04',
    'May': '05',
    'Jun': '06',
    'Jul': '07',
    'Aug': '08',
    'Sep': '09',
    'Oct': '10',
    'Nov': '11',
    'Dec': '12'
}


df_WIPP_columns = ["Province", "Plant_Code", "Plantname", "Statistical month", "Capacity_MW","Monthly_Gen_kWh"]
df_WIPP = pd.DataFrame(columns=df_WIPP_columns)


for index, row in df_wasteinplant.iterrows():
    province = row['Pro_en']
    plantname = row['Name']
    plantcode = row['Plant_Code']
    capacity = row['Capacity_elec_MW']
    annual_gen = row['Annual_Gen_kWh']

    for month in month_dict.keys():
        date_str = "2020" + month_dict[month] #生成日期字符串
        monthly_gen = annual_gen * df_load.loc[df_load["Province"] == province, month].values[0] #获取特定省份、月份的发电量比例，计算月发电量
        df_WIPP.loc[len(df_WIPP)] = [province, plantcode, plantname, date_str, capacity, monthly_gen]
        


### 计算月垃圾焚烧量
### 垃圾量（t）* 垃圾热值（GJ/t）*转换效率（%）* 278 = 发电量（kWh） 1 GJ = 278 kWh

ProcessEfficiency_WIPP = 0.2 #转换效率
CalorificValue_Waste = 6.82 #垃圾热值 GJ/t
Monthly_Waste = df_WIPP['Monthly_Gen_kWh']/278/CalorificValue_Waste/ProcessEfficiency_WIPP
df_WIPP['Monthly_Waste_t'] = Monthly_Waste

#数据说明与来源：1557-1700 kcal/kg = 6.52-7.12 GJ/t （参考文献：谷琳,何坤,刘海威.中国生活垃圾焚烧发电项目垃圾热值特性及其影响研究[C]）

df_WIPP.to_excel(os.path.join(folder_path, "WIPP.xlsx"), index=False)

In [11]:
# 月供热量

## 时间精度年→月 计算每个煤电厂的月供热量及供热煤耗
month_dict = {
    'Jan': '01',
    'Feb': '02',
    'Mar': '03',
    'Apr': '04',
    'May': '05',
    'Jun': '06',
    'Jul': '07',
    'Aug': '08',
    'Sep': '09',
    'Oct': '10',
    'Nov': '11',
    'Dec': '12'
}

monthly_heat_values = []

for index, row in df_coalfpplant.iterrows():
    province = row['Province']
    annual_heat = row['Annual_Heat_GJ']

    for month in month_dict.keys():
        monthly_heat = annual_heat * df_load.loc[df_load["Province"] == province, month].values[0] #获取特定省份、月份的发电量比例，计算月发电量
        monthly_heat_values.append(monthly_heat)
        
df_CFPP['Monthly_Heat_GJ'] = monthly_heat_values

### 计算月供热煤耗量、水耗量
provincecoalcon_dict = dict(zip(df_coalcon['Province'], df_coalcon['Standard_coal_con_heat(kgce/GJ)']))
Monthly_Coal = df_CFPP.apply(lambda row: row['Monthly_Heat_GJ'] * provincecoalcon_dict.get(row['Province'], 0), axis=1) / 1e3 #将kgce转化为tce
df_CFPP['Monthly_Coal_Heat_tce'] = Monthly_Coal
df_CFPP.to_excel(os.path.join(folder_path, "CFPP.xlsx"), index=False)

In [13]:
# 月供热量

## 时间精度年→月 计算每个煤电厂的月供热量及供热煤耗
month_dict = {
    'Jan': '01',
    'Feb': '02',
    'Mar': '03',
    'Apr': '04',
    'May': '05',
    'Jun': '06',
    'Jul': '07',
    'Aug': '08',
    'Sep': '09',
    'Oct': '10',
    'Nov': '11',
    'Dec': '12'
}

monthly_heat_values = []

for index, row in df_coal2.iterrows():
    province = row['Province']
    annual_heat = row['Annual_Heat_GJ']

    for month in month_dict.keys():
        monthly_heat = annual_heat * df_load.loc[df_load["Province"] == province, month].values[0] #获取特定省份、月份的发电量比例，计算月发电量
        monthly_heat_values.append(monthly_heat)
        
df_CFPP_2['Monthly_Heat_GJ'] = monthly_heat_values

### 计算月供热水耗量
pro_water = dict(zip(df_water_int['Province'],df_water_int['heat_WC_m3/GJ']))
Monthly_Water = df_CFPP_2.apply(lambda row: row['Monthly_Heat_GJ'] * pro_water.get(row['Province'], 0), axis=1)
df_CFPP_2['Monthly_Water_Heat_m3'] = Monthly_Water

df_CFPP_2.to_excel(os.path.join(folder_path, "CFPP_WITHOUT_OT_2.xlsx"), index=False)

In [17]:
## 时间精度年→月 计算每个垃圾焚烧厂的月发电量(kWh)及垃圾焚烧量(t)
month_dict = {
    'Jan': '01',
    'Feb': '02',
    'Mar': '03',
    'Apr': '04',
    'May': '05',
    'Jun': '06',
    'Jul': '07',
    'Aug': '08',
    'Sep': '09',
    'Oct': '10',
    'Nov': '11',
    'Dec': '12'
}

monthly_heat_values = []
for index, row in df_wasteinplant.iterrows():
    province = row['Pro_en']
    annual_heat = row['Annual_Heat_GJ']

    for month in month_dict.keys():
        monthly_heat = annual_heat * df_load.loc[df_load["Province"] == province, month].values[0] #获取特定省份、月份的发电量比例，计算月发电量
        monthly_heat_values.append(monthly_heat)
        
df_WIPP['Monthly_Heat_GJ'] = monthly_heat_values

### 计算月供热水耗量
Monthly_Water = df_WIPP.apply(lambda row: row['Monthly_Heat_GJ'] * pro_water.get(row['Province'], 0), axis=1)
df_WIPP['Monthly_Water_Heat_m3'] = Monthly_Water
       
# ### 计算月垃圾焚烧量
# ### 垃圾量（t）* 垃圾热值（GJ/t）*转换效率（%）* 278 = 发电量（kWh） 1 GJ = 278 kWh

# ProcessEfficiency_WIPP = 0.2 #转换效率
# CalorificValue_Waste = 6.82 #垃圾热值 GJ/t
# Monthly_Waste = df_WIPP['Monthly_Gen_kWh']/278/CalorificValue_Waste/ProcessEfficiency_WIPP
# df_WIPP['Monthly_Waste_t'] = Monthly_Waste

# #数据说明与来源：1557-1700 kcal/kg = 6.52-7.12 GJ/t （参考文献：谷琳,何坤,刘海威.中国生活垃圾焚烧发电项目垃圾热值特性及其影响研究[C]）

df_WIPP.to_excel(os.path.join(folder_path, "WIPP.xlsx"), index=False)

In [19]:
# 月供热量

## 时间精度年→月 计算每个煤电厂的月供热量及供热煤耗
month_dict = {
    'Jan': '01',
    'Feb': '02',
    'Mar': '03',
    'Apr': '04',
    'May': '05',
    'Jun': '06',
    'Jul': '07',
    'Aug': '08',
    'Sep': '09',
    'Oct': '10',
    'Nov': '11',
    'Dec': '12'
}

monthly_heat_values = []

for index, row in df_gas_plant.iterrows():
    province = row['Province']
    annual_heat = row['Annual_Heat_GJ']

    for month in month_dict.keys():
        monthly_heat = annual_heat * df_load.loc[df_load["Province"] == province, month].values[0] #获取特定省份、月份的发电量比例，计算月发电量
        monthly_heat_values.append(monthly_heat)
        
df_GFPP['Monthly_Heat_GJ'] = monthly_heat_values

### 计算月供热水耗量
Monthly_Water = df_GFPP.apply(lambda row: row['Monthly_Heat_GJ'] * pro_water.get(row['Province'], 0), axis=1)
df_GFPP['Monthly_Water_Heat_m3'] = Monthly_Water

df_GFPP.to_excel(os.path.join(folder_path, "GFPP.xlsx"), index=False)

In [20]:
# ### 垃圾量（t）* 垃圾热值（GJ/t）*转换效率（%）* 278 = 发电量（kWh） 1 GJ = 278 kWh

# ProcessEfficiency_WIPP = 0.2 #转换效率
# CalorificValue_Waste = 6.82 #垃圾热值 GJ/t
# Monthly_Waste = df_WIPP['Monthly_Gen_kWh']/278/CalorificValue_Waste/ProcessEfficiency_WIPP
# df_WIPP['Monthly_Waste_t'] = Monthly_Waste

gen = df_WIPP['Monthly_Gen_kWh'].sum()
heat = df_WIPP['Monthly_Heat_GJ'].sum()
waste_amount = 14607.6*10000 # t
cv = (heat+gen*0.0036)/0.2/waste_amount

In [22]:
ProcessEfficiency_WIPP = 0.2 #转换效率
# CalorificValue_Waste = 11.39 #垃圾热值 GJ/t
Monthly_Waste = df_WIPP['Monthly_Gen_kWh']/278/cv/ProcessEfficiency_WIPP
Monthly_Waste_Heat = df_WIPP['Monthly_Heat_GJ']/cv/ProcessEfficiency_WIPP
df_WIPP['Monthly_Waste_Ele_t'] = Monthly_Waste
df_WIPP['Monthly_Waste_Heat_t'] = Monthly_Waste_Heat
df_WIPP.to_excel(os.path.join(folder_path, "WIPP.xlsx"), index=False)

### 计算各省月总供电量

In [6]:
gen_p = dict(zip(df_thermal_gen['province'],df_thermal_gen['gen_total_1e8kWh']))

df_load = df_load[df_load['Province']!= 'West_Inner_Mongolia']

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for month in months:
    df_load[month] = df_load.apply(lambda row: row[month] * gen_p[row['Province']], axis=1)

df_load

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_39636\1678771514.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_load[month] = df_load.apply(lambda row: row[month] * gen_p[row['Province']], axis=1)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_39636\1678771514.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_load[month] = df_load.apply(lambda row: row[month] * gen_p[row['Province']], axis=1)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_39636\1678771514.py:7: SettingWithCopyWarning: 
A value is tryi

,Province,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,Beijing,45.409949,35.732419,32.754717,27.543739,30.149228,35.360206,42.432247,38.710120,32.754717,29.032590,37.593482,46.526587
1,Tianjin,60.865604,47.170843,54.018223,48.692483,51.735763,54.779043,64.669704,58.583144,53.257403,50.974943,57.822323,65.430524
2,Hebei,210.544890,163.553710,187.354438,184.303061,189.185263,201.390762,219.088739,209.934614,184.303061,174.538660,194.067462,233.735339
3,Shanxi,268.873563,222.337755,237.849689,232.679046,239.142352,241.727674,254.654289,241.727674,223.630415,230.093724,252.068967,279.214854
4,Inner Mongolia,396.300575,354.857379,391.120177,379.464275,383.349576,379.464275,400.185875,407.956472,396.300575,404.071176,407.956472,429.973174
6,Jilin,65.490833,53.774583,62.186250,56.778750,57.379583,55.877500,61.885833,60.383750,56.177917,58.581250,63.087500,69.396250
7,Liaoning,131.961539,110.269230,117.500000,110.269230,110.269230,106.653847,117.500000,113.884616,106.653847,113.884616,128.346154,142.807692
8,Heilongjiang,81.772088,69.191767,75.481927,72.786144,72.786144,68.293173,71.887550,69.191767,66.495984,75.481927,81.772088,89.859437
9,Shanghai,79.346301,61.772959,67.630740,59.110331,59.642857,64.968112,82.541454,90.529336,70.293367,62.305485,60.707908,76.151148
10,Jiangsu,380.724660,288.282522,351.280129,325.944136,330.737430,344.432560,415.647246,425.918596,356.758180,338.954509,343.747807,387.572225


In [8]:
df_load.to_csv(r"E:\Test\Inputs\load_amount_monthly_province.csv",index=False)